In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import os, gc, warnings
import random
import datetime
import warnings

from tqdm.notebook import tqdm

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from sklearn.linear_model import LinearRegression

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold
import sklearn
import category_encoders

import missingno as msno

import lightgbm as lgb

import pickle

warnings.filterwarnings('ignore')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#data import
path = "E:/pasca/Documents/Test/"
train_df = pd.read_csv(path + 'train.csv')
building_df = pd.read_csv(path + 'building_metadata.csv')
weather_df = pd.read_csv(path + 'weather_train.csv')

In [3]:
# Original code from https://www.kaggle.com/aitude/ashrae-missing-weather-data-handling by @aitude

def fill_weather_dataset(weather_df):
    
    # Find Missing Dates
    time_format = "%Y-%m-%d %H:%M:%S"
    start_date = datetime.datetime.strptime(weather_df['timestamp'].min(),time_format)
    end_date = datetime.datetime.strptime(weather_df['timestamp'].max(),time_format)
    total_hours = int(((end_date - start_date).total_seconds() + 3600) / 3600)
    hours_list = [(end_date - datetime.timedelta(hours=x)).strftime(time_format) for x in range(total_hours)]

    missing_hours = []
    for site_id in range(16):
        site_hours = np.array(weather_df[weather_df['site_id'] == site_id]['timestamp'])
        new_rows = pd.DataFrame(np.setdiff1d(hours_list,site_hours),columns=['timestamp'])
        new_rows['site_id'] = site_id
        weather_df = pd.concat([weather_df,new_rows])

        weather_df = weather_df.reset_index(drop=True)           

    # Add new Features
    weather_df["datetime"] = pd.to_datetime(weather_df["timestamp"])
    weather_df["day"] = weather_df["datetime"].dt.day
    weather_df["week"] = weather_df["datetime"].dt.week
    weather_df["month"] = weather_df["datetime"].dt.month
    
    # Reset Index for Fast Update
    weather_df = weather_df.set_index(['site_id','day','month'])

    air_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['air_temperature'].mean(),columns=["air_temperature"])
    weather_df.update(air_temperature_filler,overwrite=False)

    # Step 1
    cloud_coverage_filler = weather_df.groupby(['site_id','day','month'])['cloud_coverage'].mean()
    # Step 2
    cloud_coverage_filler = pd.DataFrame(cloud_coverage_filler.fillna(method='ffill'),columns=["cloud_coverage"])

    weather_df.update(cloud_coverage_filler,overwrite=False)

    due_temperature_filler = pd.DataFrame(weather_df.groupby(['site_id','day','month'])['dew_temperature'].mean(),columns=["dew_temperature"])
    weather_df.update(due_temperature_filler,overwrite=False)

    # Step 1
    sea_level_filler = weather_df.groupby(['site_id','day','month'])['sea_level_pressure'].mean()
    # Step 2
    sea_level_filler = pd.DataFrame(sea_level_filler.fillna(method='ffill'),columns=['sea_level_pressure'])

    weather_df.update(sea_level_filler,overwrite=False)

    wind_direction_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_direction'].mean(),columns=['wind_direction'])
    weather_df.update(wind_direction_filler,overwrite=False)

    wind_speed_filler =  pd.DataFrame(weather_df.groupby(['site_id','day','month'])['wind_speed'].mean(),columns=['wind_speed'])
    weather_df.update(wind_speed_filler,overwrite=False)

    # Step 1
    precip_depth_filler = weather_df.groupby(['site_id','day','month'])['precip_depth_1_hr'].mean()
    # Step 2
    precip_depth_filler = pd.DataFrame(precip_depth_filler.fillna(method='ffill'),columns=['precip_depth_1_hr'])

    weather_df.update(precip_depth_filler,overwrite=False)

    weather_df = weather_df.reset_index()
    weather_df = weather_df.drop(['datetime','day','week'],axis=1)
        
    return weather_df

# Original code from https://www.kaggle.com/gemartin/load-data-reduce-memory-usage by @gemartin

from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

def reduce_mem_usage(df, use_float16=False):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    
    start_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage of dataframe is {:.2f} MB".format(start_mem))
    
    for col in df.columns:
        if is_datetime(df[col]) or is_categorical_dtype(df[col]):
            continue
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if use_float16 and c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print("Memory usage after optimization is: {:.2f} MB".format(end_mem))
    print("Decreased by {:.1f}%".format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def features_engineering(df):
    
    # Sort by timestamp
    df.sort_values("timestamp")
    df.reset_index(drop=True)
    
    # Add more features
    df["timestamp"] = pd.to_datetime(df["timestamp"],format="%Y-%m-%d %H:%M:%S")
    df["hour"] = df["timestamp"].dt.hour
    df["weekend"] = df["timestamp"].dt.weekday
    holidays = ["2016-01-01", "2016-01-18", "2016-02-15", "2016-05-30", "2016-07-04",
                    "2016-09-05", "2016-10-10", "2016-11-11", "2016-11-24", "2016-12-26",
                    "2017-01-02", "2017-01-16", "2017-02-20", "2017-05-29", "2017-07-04",
                    "2017-09-04", "2017-10-09", "2017-11-10", "2017-11-23", "2017-12-25",
                    "2018-01-01", "2018-01-15", "2018-02-19", "2018-05-28", "2018-07-04",
                    "2018-09-03", "2018-10-08", "2018-11-12", "2018-11-22", "2018-12-25",
                    "2019-01-01"]
    df["is_holiday"] = (df.timestamp.isin(holidays)).astype(int)
    df['square_feet'] =  np.log1p(df['square_feet']**0.5)
    
    # Remove Unused Columns
    #drop = ["timestamp","sea_level_pressure", "wind_direction", "wind_speed","year_built","floor_count"]
    #df = df.drop(drop, axis=1)
    #gc.collect()
    
    # Encode Categorical Data
    le = LabelEncoder()
    df["primary_use"] = le.fit_transform(df["primary_use"])
    
    return df

In [4]:
weather_df = fill_weather_dataset(weather_df)

In [5]:
train_df = reduce_mem_usage(train_df,use_float16=True)
building_df = reduce_mem_usage(building_df,use_float16=True)
weather_df = reduce_mem_usage(weather_df,use_float16=True)

Memory usage of dataframe is 616.95 MB
Memory usage after optimization is: 173.84 MB
Decreased by 71.8%
Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.9%
Memory usage of dataframe is 10.72 MB
Memory usage after optimization is: 2.73 MB
Decreased by 74.5%


In [6]:
train_df = train_df.merge(building_df, left_on='building_id',right_on='building_id',how='left')
train_df = train_df.merge(weather_df,how='left',left_on=['site_id','timestamp'],right_on=['site_id','timestamp'])

In [7]:
train_df = features_engineering(train_df)

In [8]:
train_df.drop(train_df.loc[train_df['building_id']== 1099].index, inplace=True)
train_df.drop(train_df.loc[train_df['building_id']== 778].index, inplace=True)

In [9]:
del train_df["year_built"]
del train_df["floor_count"]

In [10]:
mask = train_df["meter"] == 0
train_df_0 = train_df[mask]

mask = train_df["meter"] == 1
train_df_1 = train_df[mask]

mask = train_df["meter"] == 2
train_df_2 = train_df[mask]

mask = train_df["meter"] == 3
train_df_3 = train_df[mask]

In [11]:
#target = train_df_0["meter_reading"]
target = np.log1p(train_df_0["meter_reading"])
train = train_df_0.drop(["building_id", 'meter', 'meter_reading', 'timestamp','hour', "weekend", "is_holiday"], axis = 1)
train.head()

site_id  primary_use  square_feet  month  air_temperature  cloud_coverage  \
0        0            0     4.468308      1             25.0             6.0   
1        0            0     3.973186      1             25.0             6.0   
2        0            0     4.308396      1             25.0             6.0   
3        0            0     5.042775      1             25.0             6.0   
4        0            0     5.836206      1             25.0             6.0   

   dew_temperature  precip_depth_1_hr  sea_level_pressure  wind_direction  \
0             20.0           -0.17395              1019.5             0.0   
1             20.0           -0.17395              1019.5             0.0   
2             20.0           -0.17395              1019.5             0.0   
3             20.0           -0.17395              1019.5             0.0   
4             20.0           -0.17395              1019.5             0.0   

   wind_speed  
0         0.0  
1         0.0  
2         0.0  
3         0.0  
4         0.0

In [12]:
train_1 = pd.get_dummies(train, columns = ["site_id", "primary_use",'month'])

In [13]:
train_1.sample(5)

square_feet  air_temperature  cloud_coverage  dew_temperature  \
1554940     5.623867        -0.600098        2.000000        -6.699219   
1883002     4.633824        -8.898438        4.855469       -15.000000   
3494869     5.336166        23.296875        2.000000         2.800781   
6038712     6.063940         7.199219        0.000000         0.600098   
5278692     6.803215        25.593750        4.000000         6.101562   

         precip_depth_1_hr  sea_level_pressure  wind_direction  wind_speed  \
1554940               -1.0              1002.0          170.00    5.101562   
1883002                0.0              1020.0          260.00    1.500000   
3494869                0.0              1020.5          204.75    2.599609   
6038712                0.0              1020.0          100.00    3.099609   
5278692                0.0              1010.5          280.00    2.099609   

         site_id_0  site_id_1  ...  month_3  month_4  month_5  month_6  \
1554940          0          0  ...        0        0        0        0   
1883002          0          0  ...        0        0        0        0   
3494869          1          0  ...        1        0        0        0   
6038712          0          0  ...        0        1        0        0   
5278692          0          0  ...        0        1        0        0   

         month_7  month_8  month_9  month_10  month_11  month_12  
1554940        0        0        0         0         0         0  
1883002        0        0        0         0         0         0  
3494869        0        0        0         0         0         0  
6038712        0        0        0         0         0         0  
5278692        0        0        0         0         0         0  

[5 rows x 52 columns]

In [14]:
model = LinearRegression().fit(train_1, target)

In [15]:
r_sq = model.score(train_1, target)
print(r_sq)

0.4753870674960864


In [16]:
#data import
path = "E:/pasca/Documents/Test/"
test_df = pd.read_csv(path + 'test.csv')
building_test_df = pd.read_csv(path + 'building_metadata.csv')
weather_test_df = pd.read_csv(path + 'weather_test.csv')

In [17]:
weather_test_df = fill_weather_dataset(weather_test_df)

In [18]:
row_ids = test_df["row_id"]
test_df.drop("row_id", axis=1, inplace=True)

In [19]:
test_df = reduce_mem_usage(test_df,use_float16=True)
building_test_df = reduce_mem_usage(building_test_df,use_float16=True)
weather_test_df = reduce_mem_usage(weather_test_df,use_float16=True)

Memory usage of dataframe is 954.38 MB
Memory usage after optimization is: 199.47 MB
Decreased by 79.1%
Memory usage of dataframe is 0.07 MB
Memory usage after optimization is: 0.02 MB
Decreased by 73.9%
Memory usage of dataframe is 21.39 MB
Memory usage after optimization is: 5.45 MB
Decreased by 74.5%


In [20]:
weather_test_df.head()

site_id  month            timestamp  air_temperature  cloud_coverage  \
0        0      1  2017-01-01 00:00:00        17.796875             4.0   
1        0      1  2017-01-01 01:00:00        17.796875             2.0   
2        0      1  2017-01-01 02:00:00        16.093750             0.0   
3        0      1  2017-01-01 03:00:00        17.203125             0.0   
4        0      1  2017-01-01 04:00:00        16.703125             2.0   

   dew_temperature  precip_depth_1_hr  sea_level_pressure  wind_direction  \
0        11.703125           0.282715              1021.5           100.0   
1        12.796875           0.000000              1022.0           130.0   
2        12.796875           0.000000              1022.0           140.0   
3        13.296875           0.000000              1022.0           140.0   
4        13.296875           0.000000              1022.5           130.0   

   wind_speed  
0    3.599609  
1    3.099609  
2    3.099609  
3    3.099609  
4    2.599609

In [21]:
building_test_df.head()

site_id  building_id primary_use  square_feet  year_built  floor_count
0        0            0   Education         7432      2008.0          NaN
1        0            1   Education         2720      2004.0          NaN
2        0            2   Education         5376      1991.0          NaN
3        0            3   Education        23685      2002.0          NaN
4        0            4   Education       116607      1975.0          NaN

In [22]:
test_df.head()

building_id  meter            timestamp
0            0      0  2017-01-01 00:00:00
1            1      0  2017-01-01 00:00:00
2            2      0  2017-01-01 00:00:00
3            3      0  2017-01-01 00:00:00
4            4      0  2017-01-01 00:00:00

In [23]:
test_df = test_df.merge(building_test_df, left_on='building_id',right_on='building_id',how='left')
test_df = test_df.merge(weather_test_df,how='left',left_on=['site_id','timestamp'],right_on=['site_id','timestamp'])

In [24]:
test_df.head()

building_id  meter            timestamp  site_id primary_use  square_feet  \
0            0      0  2017-01-01 00:00:00        0   Education         7432   
1            1      0  2017-01-01 00:00:00        0   Education         2720   
2            2      0  2017-01-01 00:00:00        0   Education         5376   
3            3      0  2017-01-01 00:00:00        0   Education        23685   
4            4      0  2017-01-01 00:00:00        0   Education       116607   

   year_built  floor_count  month  air_temperature  cloud_coverage  \
0      2008.0          NaN      1        17.796875             4.0   
1      2004.0          NaN      1        17.796875             4.0   
2      1991.0          NaN      1        17.796875             4.0   
3      2002.0          NaN      1        17.796875             4.0   
4      1975.0          NaN      1        17.796875             4.0   

   dew_temperature  precip_depth_1_hr  sea_level_pressure  wind_direction  \
0        11.703125           0.282715              1021.5           100.0   
1        11.703125           0.282715              1021.5           100.0   
2        11.703125           0.282715              1021.5           100.0   
3        11.703125           0.282715              1021.5           100.0   
4        11.703125           0.282715              1021.5           100.0   

   wind_speed  
0    3.599609  
1    3.599609  
2    3.599609  
3    3.599609  
4    3.599609

In [25]:
test_df = features_engineering(test_df)

In [26]:
#test_df.drop(test_df.loc[test_df['building_id']== 1099].index, inplace=True)
#test_df.drop(test_df.loc[test_df['building_id']== 778].index, inplace=True)

In [27]:
test_df.head()

building_id  meter  timestamp  site_id  primary_use  square_feet  \
0            0      0 2017-01-01        0            0     4.468308   
1            1      0 2017-01-01        0            0     3.973186   
2            2      0 2017-01-01        0            0     4.308396   
3            3      0 2017-01-01        0            0     5.042775   
4            4      0 2017-01-01        0            0     5.836206   

   year_built  floor_count  month  air_temperature  cloud_coverage  \
0      2008.0          NaN      1        17.796875             4.0   
1      2004.0          NaN      1        17.796875             4.0   
2      1991.0          NaN      1        17.796875             4.0   
3      2002.0          NaN      1        17.796875             4.0   
4      1975.0          NaN      1        17.796875             4.0   

   dew_temperature  precip_depth_1_hr  sea_level_pressure  wind_direction  \
0        11.703125           0.282715              1021.5           100.0   
1        11.703125           0.282715              1021.5           100.0   
2        11.703125           0.282715              1021.5           100.0   
3        11.703125           0.282715              1021.5           100.0   
4        11.703125           0.282715              1021.5           100.0   

   wind_speed  hour  weekend  is_holiday  
0    3.599609     0        6           0  
1    3.599609     0        6           0  
2    3.599609     0        6           0  
3    3.599609     0        6           0  
4    3.599609     0        6           0

In [28]:
del test_df["year_built"]
del test_df["floor_count"]

In [29]:
mask = test_df["meter"] == 0
test_df_0 = test_df[mask]

mask = test_df["meter"] == 1
test_df_1 = test_df[mask]

mask = test_df["meter"] == 2
test_df_2 = test_df[mask]

mask = test_df["meter"] == 3
test_df_3 = test_df[mask]

In [30]:
test = test_df_0.drop(["building_id", 'meter', 'timestamp','hour', "weekend", "is_holiday"], axis = 1)
test.head()

site_id  primary_use  square_feet  month  air_temperature  cloud_coverage  \
0        0            0     4.468308      1        17.796875             4.0   
1        0            0     3.973186      1        17.796875             4.0   
2        0            0     4.308396      1        17.796875             4.0   
3        0            0     5.042775      1        17.796875             4.0   
4        0            0     5.836206      1        17.796875             4.0   

   dew_temperature  precip_depth_1_hr  sea_level_pressure  wind_direction  \
0        11.703125           0.282715              1021.5           100.0   
1        11.703125           0.282715              1021.5           100.0   
2        11.703125           0.282715              1021.5           100.0   
3        11.703125           0.282715              1021.5           100.0   
4        11.703125           0.282715              1021.5           100.0   

   wind_speed  
0    3.599609  
1    3.599609  
2    3.599609  
3    3.599609  
4    3.599609

In [31]:
test = pd.get_dummies(test, columns = ["site_id", "primary_use",'month'])

In [44]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24755760 entries, 0 to 41697599
Data columns (total 52 columns):
 #   Column              Dtype  
---  ------              -----  
 0   square_feet         float64
 1   air_temperature     float16
 2   cloud_coverage      float16
 3   dew_temperature     float16
 4   precip_depth_1_hr   float16
 5   sea_level_pressure  float16
 6   wind_direction      float16
 7   wind_speed          float16
 8   site_id_0           uint8  
 9   site_id_1           uint8  
 10  site_id_2           uint8  
 11  site_id_3           uint8  
 12  site_id_4           uint8  
 13  site_id_5           uint8  
 14  site_id_6           uint8  
 15  site_id_7           uint8  
 16  site_id_8           uint8  
 17  site_id_9           uint8  
 18  site_id_10          uint8  
 19  site_id_11          uint8  
 20  site_id_12          uint8  
 21  site_id_13          uint8  
 22  site_id_14          uint8  
 23  site_id_15          uint8  
 24  primary_use_0       ui

In [50]:
print(row_ids.count())

41697600


In [40]:
results = np.expm1(model.predict(test))


In [42]:
results_df = pd.DataFrame({"meter_reading": results})

In [43]:
results_df.to_csv("submission_lr_new_3.csv", index=False)